# La calidad de los datos suele no ser la mejor.

### Un ejemplo tomando datos de U.S. Energy Information Administration

Fuente de los datos: [eia.gov](http://www.eia.gov/coal/data.cfm)

¿Que hacer?

1. **Revisar los datos** Siempre asuman que los datos tienen errores.
2. **Limpiar y estandarizar** Es necesario procesar los datos para que sean homogeneos y confiables.

In [1]:
%matplotlib inline

In [2]:
import asyncio
import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
#import seaborn as sns
import xlrd

from IPython.display import IFrame
from xml.sax import ContentHandler, parse

In [3]:
# Veamos como se ve la fuente de los datos metiendola en un IFrame
IFrame("http://www.eia.gov/coal/data.cfm", width=950, height=600)

In [37]:
# Funcion para manejar la descarga de archivos y guardarlos apropiadamente.
def download_file(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url, stream=True)
    with open(f"big_data_chunks/uncleaned_coal_data/{local_filename}", 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk:
                f.write(chunk)

    return local_filename

In [38]:
# No vamos a escribir a mano la lista de todos los archivos, para ello usamos la dirección tipo de los archivos.
# https://www.eia.gov/coal/data/public/xls/coalpublic1985.xls y descargamos.
for year in range(1983, 2016):
    print(f"Downloading year {year}")
    download_file(f"https://www.eia.gov/coal/data/public/xls/coalpublic{year}.xls")
    print(f"Year {year} downloaded")

Year 1985 downloaded
Year 1986 downloaded
Year 1987 downloaded
Year 1988 downloaded
Year 1989 downloaded
Year 1990 downloaded
Year 1991 downloaded
Year 1992 downloaded
Year 1993 downloaded
Year 1994 downloaded
Year 1995 downloaded
Year 1996 downloaded
Year 1997 downloaded
Year 1998 downloaded
Year 1999 downloaded
Year 2000 downloaded
Year 2001 downloaded
Year 2002 downloaded
Year 2003 downloaded
Year 2004 downloaded
Year 2005 downloaded
Year 2006 downloaded
Year 2007 downloaded
Year 2008 downloaded
Year 2009 downloaded
Year 2010 downloaded
Year 2011 downloaded
Year 2012 downloaded
Year 2013 downloaded
Year 2014 downloaded
Year 2015 downloaded


In [39]:
# Un ligero analisis de los datos descargados.
try:
    df_test = pd.read_excel('big_data_chunks/uncleaned_coal_data/coalpublic1986.xls', skiprows=3)
    df_test.head()

except xlrd.XLRDError as e:
    print("Upppssie", e)

Upppssie Unsupported format, or corrupt file: Expected BOF record; found b'<?xml ve'


In [40]:
# ¿Que paso? Las fuentes de datos parece que tienen problemas.
# Veamos otro archivo.
df_test = pd.read_excel("big_data_chunks/uncleaned_coal_data/coalpublic2016.xls", skiprows=3)
df_test.head()

,Year,MSHA ID,Mine Name,Mine State,Mine County,Mine Status,Mine Type,Company Type,Operation Type,Operating Company,Operating Company Address,Union Code,Coal Supply Region,Production (short tons),Average Employees,Labor Hours
0,2016,103381,Jesse Creek H.W.M.,Alabama,Bibb,Active,Surface,Indepedent Producer Operator,Mine only,"Jesse Creek Mining, Llc","1615 Kent Dairy Rd, Alabaster, AL 35007",NaN,Appalachia Southern,37786,6,14235
1,2016,103246,Bear Creek,Alabama,Franklin,Active,Surface,Indepedent Producer Operator,Mine only,"Yellowhammer Energy Solutions,","912 Edenton Street, Birmingham, AL 35242",NaN,Appalachia Southern,43155,21,13278
2,2016,103451,Knight Mine,Alabama,Franklin,Temporarily closed,Surface,Indepedent Producer Operator,Mine only,"Birmingham Coal & Coke Co., In","P.O. Box 354, Lynn, AL 35242",NaN,Appalachia Southern,22218,19,11158
3,2016,100329,Concord Mine,Alabama,Jefferson,Active,Underground,Operating Subsidiary,Preparation Plant,Oak Grove Resources Llc,"8360 Taylors Ferry Rd, Hueytown, AL 35023",United Mine Workers Of America,Appalachia Southern,0,43,93811
4,2016,100627,Flat Top Mine,Alabama,Jefferson,Active,Surface,Indepedent Producer Operator,Mine only,"Cedar Lake Mining, Inc","7802 Highway 78, Cordova, AL 35550",NaN,Appalachia Southern,281668,46,126268


In [44]:
# Tomado sin mucha vergüenza de aquí https://www.oreilly.com/library/view/python-cookbook-2nd/0596007973/ch12s08.html
class ExcelHandler(ContentHandler):
    def __init__(self):
        self.chars = []
        self.cells = []
        self.rows = []
        self.tables = []
        
    def characters(self, content):
        self.chars.append(content)
        
    def startElement(self, name, atts):
        if name=="Cell":
            self.chars = []
            
        elif name=="Row":
            self.cells=[]
            
        elif name=="Table":
            self.rows = []
            
    def endElement(self, name):
        if name=="Cell":
            self.cells.append(''.join(self.chars))
            
        elif name=="Row":
            self.rows.append(self.cells)
            
        elif name=="Table":
            self.tables.append(self.rows)


def create_file(org_file_addr, df):
    new_addr = 'big_data_chunks/cleaned_coal_data/coal_prod_' + org_file_addr[-8:-4] + '.xlsx'
    df.to_excel(new_addr)

    
def transform_xlrdfiles(file_name):
    excel_handler = ExcelHandler()
    parse(file_name, excel_handler)
    df = pd.DataFrame(excel_handler.tables[0][4:], columns=excel_handler.tables[0][3])
    create_file(file_name, df)

In [46]:
unclean_list = glob.glob('big_data_chunks/uncleaned_coal_data/*.xls')
for file in unclean_list:
    print('Procesando:', file)
    try:
        df = pd.read_excel(file, skiprows=3)
        create_file(file, df)
        print(' ...Ok')

    except:
        transform_xlrdfiles(file)
        print(' ...Ok')
    
print('\n¡Proceso Terminado!')

Procesando: big_data_chunks/uncleaned_coal_data/coalpublic2009.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1996.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1999.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic2012.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic2003.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1994.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic2007.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic2010.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1993.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1986.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1990.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic1995.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_coal_data/coalpublic2001.xls
 ...Ok
Procesando: big_data_chunks/uncleaned_

In [49]:
clean_list = glob.glob('big_data_chunks/cleaned_coal_data/*.xlsx')
df = pd.DataFrame()
for el in clean_list:
    df = pd.concat((df, pd.read_excel(el, index_col=False, sort=True)))
    

/home/maestro/.virtualenvs/datas/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


In [ ]:
df1 = pd.read_excel("cleaned_data/coal_prod_2001.xlsx", index_col=False)
df2 = pd.read_excel("cleaned_data/coal_prod_2002.xlsx", index_col=False)
df3 = pd.read_excel("cleaned_data/coal_prod_2003.xlsx", index_col=False)
df4 = pd.read_excel("cleaned_data/coal_prod_2004.xlsx", index_col=False)
df5 = pd.read_excel("cleaned_data/coal_prod_2005.xlsx", index_col=False)
df6 = pd.read_excel("cleaned_data/coal_prod_2006.xlsx", index_col=False)
df7 = pd.read_excel("cleaned_data/coal_prod_2007.xlsx", index_col=False)
df8 = pd.read_excel("cleaned_data/coal_prod_2008.xlsx", index_col=False)
df9 = pd.read_excel("cleaned_data/coal_prod_2009.xlsx", index_col=False)
df10 = pd.read_excel("cleaned_data/coal_prod_2010.xlsx", index_col=False)
df11 = pd.read_excel("cleaned_data/coal_prod_2011.xlsx", index_col=False)
df12 = pd.read_excel("cleaned_data/coal_prod_2012.xlsx", index_col=False)
df13 = pd.read_excel("cleaned_data/coal_prod_2013.xlsx", index_col=False)
df14 = pd.read_excel("cleaned_data/coal_prod_2014.xlsx", index_col=False)
df15 = pd.read_excel("cleaned_data/coal_prod_2015.xlsx", index_col=False)
df = pd.concat((df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11, df12, df13, df14, df15))

In [51]:
df

,Average Employees,Coal Supply Region,Company Type,Labor Hours,MSHA ID,Mine Basin,Mine County,Mine Name,Mine State,Mine Status,Mine Type,Operating Company,Operating Company Address,Operation Type,Production (short tons),Union Code,Year
0,37786.0,NaN,Surface,6,2016,NaN,Alabama,103381,Jesse Creek H.W.M.,Bibb,Active,Mine only,"Jesse Creek Mining, Llc",Indepedent Producer Operator,Appalachia Southern,"1615 Kent Dairy Rd, Alabaster, AL 35007",0
1,43155.0,NaN,Surface,21,2016,NaN,Alabama,103246,Bear Creek,Franklin,Active,Mine only,"Yellowhammer Energy Solutions,",Indepedent Producer Operator,Appalachia Southern,"912 Edenton Street, Birmingham, AL 35242",1
2,22218.0,NaN,Surface,19,2016,NaN,Alabama,103451,Knight Mine,Franklin,Temporarily closed,Mine only,"Birmingham Coal & Coke Co., In",Indepedent Producer Operator,Appalachia Southern,"P.O. Box 354, Lynn, AL 35242",2
3,0.0,United Mine Workers Of America,Underground,43,2016,NaN,Alabama,100329,Concord Mine,Jefferson,Active,Preparation Plant,Oak Grove Resources Llc,Operating Subsidiary,Appalachia Southern,"8360 Taylors Ferry Rd, Hueytown, AL 35023",3
4,281668.0,NaN,Surface,46,2016,NaN,Alabama,100627,Flat Top Mine,Jefferson,Active,Mine only,"Cedar Lake Mining, Inc",Indepedent Producer Operator,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550",4
5,1474588.0,United Mine Workers Of America,Underground,303,2016,NaN,Alabama,100851,Oak Grove Mine,Jefferson,Active,Mine only,Oak Grove Resources Llc,Operating Subsidiary,Appalachia Southern,"8360 Taylors Ferry Rd, Hueytown, AL 35023",5
6,3067565.0,United Mine Workers Of America,Underground,539,2016,NaN,Alabama,101401,No 7 Mine,Jefferson,Active,Mine and Preparation Plant,"Warrior Met Coal Mining, Llc",Operating Subsidiary,Appalachia Southern,"P.O. Box 133, Brookwood, AL 35444",6
7,0.0,NaN,Surface,7,2016,NaN,Alabama,103101,Pratt No. 1 Mine,Jefferson,Active,Preparation Plant,"Cane Creek, Llc",Operating Subsidiary,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550",7
8,234852.0,NaN,Surface,54,2016,NaN,Alabama,103180,Sloan Mountain Mine,Jefferson,Active,Mine only,M S & R Equipment Company Inc,Indepedent Producer Operator,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550",8
9,108420.0,NaN,Surface,28,2016,NaN,Alabama,103285,Narley Mine,Jefferson,Active,Mine only,Best Coal Inc,Operating Subsidiary,Appalachia Southern,"7802 Highway 78, Parrish, AL 35550",9


In [52]:
df.drop(['Coal Supply Region', 'Year'], axis=1, inplace=True)

In [53]:
df.head()

,Average Employees,Company Type,Labor Hours,MSHA ID,Mine Basin,Mine County,Mine Name,Mine State,Mine Status,Mine Type,Operating Company,Operating Company Address,Operation Type,Production (short tons),Union Code
0,37786.0,Surface,6,2016,NaN,Alabama,103381,Jesse Creek H.W.M.,Bibb,Active,Mine only,"Jesse Creek Mining, Llc",Indepedent Producer Operator,Appalachia Southern,"1615 Kent Dairy Rd, Alabaster, AL 35007"
1,43155.0,Surface,21,2016,NaN,Alabama,103246,Bear Creek,Franklin,Active,Mine only,"Yellowhammer Energy Solutions,",Indepedent Producer Operator,Appalachia Southern,"912 Edenton Street, Birmingham, AL 35242"
2,22218.0,Surface,19,2016,NaN,Alabama,103451,Knight Mine,Franklin,Temporarily closed,Mine only,"Birmingham Coal & Coke Co., In",Indepedent Producer Operator,Appalachia Southern,"P.O. Box 354, Lynn, AL 35242"
3,0.0,Underground,43,2016,NaN,Alabama,100329,Concord Mine,Jefferson,Active,Preparation Plant,Oak Grove Resources Llc,Operating Subsidiary,Appalachia Southern,"8360 Taylors Ferry Rd, Hueytown, AL 35023"
4,281668.0,Surface,46,2016,NaN,Alabama,100627,Flat Top Mine,Jefferson,Active,Mine only,"Cedar Lake Mining, Inc",Indepedent Producer Operator,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550"


In [55]:
new_columns = {
                'Average Employees': 'production_short_tons',
                'Company Type': 'mine_type',
                'Labor Hours': 'average_employees',
                'MSHA ID': 'year',
                'Mine Basin': 'mine_county',
                'Mine County': 'mine_state',
                'Mine Name': 'MSHA_ID',
                'Mine State': 'mine_name',
                'Mine Status': 'mine_basin',
                'Mine Type': 'mine_status',
                'Operating Company': 'operation_type',
                'Operating Company Address': 'operating_company',
                'Operation Type': 'company_type',
                'Production (short tons)': 'union_code',
                'Union Code': 'operating_company_address',
              }
df.rename(inplace=True, columns=new_columns)

In [56]:
df.head()

,production_short_tons,mine_type,average_employees,year,mine_county,mine_state,MSHA_ID,mine_name,mine_basin,mine_status,operation_type,operating_company,company_type,union_code,operating_company_address
0,37786.0,Surface,6,2016,NaN,Alabama,103381,Jesse Creek H.W.M.,Bibb,Active,Mine only,"Jesse Creek Mining, Llc",Indepedent Producer Operator,Appalachia Southern,"1615 Kent Dairy Rd, Alabaster, AL 35007"
1,43155.0,Surface,21,2016,NaN,Alabama,103246,Bear Creek,Franklin,Active,Mine only,"Yellowhammer Energy Solutions,",Indepedent Producer Operator,Appalachia Southern,"912 Edenton Street, Birmingham, AL 35242"
2,22218.0,Surface,19,2016,NaN,Alabama,103451,Knight Mine,Franklin,Temporarily closed,Mine only,"Birmingham Coal & Coke Co., In",Indepedent Producer Operator,Appalachia Southern,"P.O. Box 354, Lynn, AL 35242"
3,0.0,Underground,43,2016,NaN,Alabama,100329,Concord Mine,Jefferson,Active,Preparation Plant,Oak Grove Resources Llc,Operating Subsidiary,Appalachia Southern,"8360 Taylors Ferry Rd, Hueytown, AL 35023"
4,281668.0,Surface,46,2016,NaN,Alabama,100627,Flat Top Mine,Jefferson,Active,Mine only,"Cedar Lake Mining, Inc",Indepedent Producer Operator,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550"


In [57]:
df.set_index('MSHA_ID', inplace=True)

In [58]:
df.head()

,production_short_tons,mine_type,average_employees,year,mine_county,mine_state,mine_name,mine_basin,mine_status,operation_type,operating_company,company_type,union_code,operating_company_address
MSHA_ID,,,,,,,,,,,,,,
103381,37786.0,Surface,6,2016,NaN,Alabama,Jesse Creek H.W.M.,Bibb,Active,Mine only,"Jesse Creek Mining, Llc",Indepedent Producer Operator,Appalachia Southern,"1615 Kent Dairy Rd, Alabaster, AL 35007"
103246,43155.0,Surface,21,2016,NaN,Alabama,Bear Creek,Franklin,Active,Mine only,"Yellowhammer Energy Solutions,",Indepedent Producer Operator,Appalachia Southern,"912 Edenton Street, Birmingham, AL 35242"
103451,22218.0,Surface,19,2016,NaN,Alabama,Knight Mine,Franklin,Temporarily closed,Mine only,"Birmingham Coal & Coke Co., In",Indepedent Producer Operator,Appalachia Southern,"P.O. Box 354, Lynn, AL 35242"
100329,0.0,Underground,43,2016,NaN,Alabama,Concord Mine,Jefferson,Active,Preparation Plant,Oak Grove Resources Llc,Operating Subsidiary,Appalachia Southern,"8360 Taylors Ferry Rd, Hueytown, AL 35023"
100627,281668.0,Surface,46,2016,NaN,Alabama,Flat Top Mine,Jefferson,Active,Mine only,"Cedar Lake Mining, Inc",Indepedent Producer Operator,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550"


In [59]:
# Mirando los datos, creo que hay un error tipografico.
df['company_type'].unique()

array(['Indepedent Producer Operator', 'Operating Subsidiary',
       'Contractor', '-'], dtype=object)

In [60]:
# Corrijamos el valor equivocado.
df.loc[df['company_type'] == 'Indepedent Producer Operator', 'company_type'] = 'Independent Producer Operator'
df.head()

,production_short_tons,mine_type,average_employees,year,mine_county,mine_state,mine_name,mine_basin,mine_status,operation_type,operating_company,company_type,union_code,operating_company_address
MSHA_ID,,,,,,,,,,,,,,
103381,37786.0,Surface,6,2016,NaN,Alabama,Jesse Creek H.W.M.,Bibb,Active,Mine only,"Jesse Creek Mining, Llc",Independent Producer Operator,Appalachia Southern,"1615 Kent Dairy Rd, Alabaster, AL 35007"
103246,43155.0,Surface,21,2016,NaN,Alabama,Bear Creek,Franklin,Active,Mine only,"Yellowhammer Energy Solutions,",Independent Producer Operator,Appalachia Southern,"912 Edenton Street, Birmingham, AL 35242"
103451,22218.0,Surface,19,2016,NaN,Alabama,Knight Mine,Franklin,Temporarily closed,Mine only,"Birmingham Coal & Coke Co., In",Independent Producer Operator,Appalachia Southern,"P.O. Box 354, Lynn, AL 35242"
100329,0.0,Underground,43,2016,NaN,Alabama,Concord Mine,Jefferson,Active,Preparation Plant,Oak Grove Resources Llc,Operating Subsidiary,Appalachia Southern,"8360 Taylors Ferry Rd, Hueytown, AL 35023"
100627,281668.0,Surface,46,2016,NaN,Alabama,Flat Top Mine,Jefferson,Active,Mine only,"Cedar Lake Mining, Inc",Independent Producer Operator,Appalachia Southern,"7802 Highway 78, Cordova, AL 35550"


# Y para terminar:

## Un producto terminado

Un conjunto de datos limpio (mayormente), ordenado y listo para ser analizado.

In [ ]:
df.to_excel("cleaned_data/coal_prod_cleaned.xlsx")